In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math
import numpy as np
import seaborn as sns
from windrose import WindroseAxes
import matplotlib.cm as cm


Location: Latitude  56.8936   Longitude -2.9211 

Data: Nasa data https://power.larc.nasa.gov/data-access-viewer/

Data for 2001-2021

In [2]:
place = "Scotland"

file = "C:/Users/SEMORAH2/OneDrive - ABB/ABB CRC/eMine/RELIABILITY RISK/Scotland_Hourly_20010101_20211231.csv"

df = pd.read_csv(file,
                skiprows = 23) #13

df["Datetime"] = pd.to_datetime(dict(year = df.YEAR,
                                     month = df.MO,
                                     day = df.DY,
                                     hour = df.HR))

df.set_index("Datetime", inplace = True)


df = df[["WS50M","ALLSKY_SFC_SW_DWN"]]

df.columns = ["Speed_50m","GHI"]

dff=df.copy()
measured_height=50;
hub_height=60;
roughness_length=0.25; #Forest/wood lands 0.5; Few trees=0.10, many tress, few buildings 0.25
dff['Speed_hubheight'] = dff.Speed_50m*np.log(hub_height/roughness_length)/np.log(measured_height/roughness_length);
#Roughness expoenent also can be used (similar to Homer https://www.homerenergy.com/products/pro/docs/latest/how_homer_calculates_wind_turbine_power_output.html)
dff

,Speed_50m,GHI,Speed_hubheight
Datetime,,,
2001-01-01 00:00:00,10.29,0.0,10.644091
2001-01-01 01:00:00,9.50,0.0,9.826907
2001-01-01 02:00:00,9.96,0.0,10.302736
2001-01-01 03:00:00,9.82,0.0,10.157918
2001-01-01 04:00:00,10.44,0.0,10.799253
...,...,...,...
2021-12-31 19:00:00,6.70,0.0,6.930555
2021-12-31 20:00:00,6.67,0.0,6.899523
2021-12-31 21:00:00,7.02,0.0,7.261567


In [3]:
def RESCalc(n_PV):
    #######################
    #PV plant specification
    #######################
    eta_pv=0.15; #solar panel yield (%)
    area_pv=7200;#m2
    PR_PV=0.75;#Performance ratio, coefficient for losses  (range between 0.9 and 0.5, default value =  0.75)
    #######################
    #Wind farm specification
    #######################
    rated_power_wind=810000; #Wind turbine rated power in W
    eta_max_wind=0.34819; #conversion_rate Uncertainty Analysis for Wind Energy Production with Dynamic Power Curves 
    rated_speed_wind=12;
    cutin_speed_wind=2;
    cutout_speed_wind=25;
    rho_air=1.225;
    area_swept=2198;
    #######################
    #Load specification
    #######################
    Load =(5*810000)*5
    
    #######################
    #Wind turbine model
    #######################
    def power_wind(windspeed,rated_power, eta_max,rated_windspeed,cutin_speed,cutout_speed,rho,A):
        if windspeed>=cutin_speed and windspeed<rated_windspeed:
            power = 1/2*eta_max*rho*A*windspeed**3;
        elif windspeed>=rated_windspeed and windspeed<cutout_speed:
            power = rated_power;
        else:
            power = 0;
        return power #W
    
    #######################
    #PV model
    #######################
    def power_pv(GHI,eta,area,PR):
        #area_pv#m2
        #eta #solar panel yield (%)
        #PR#Performance ratio, coefficient for losses  (range between 0.9 and 0.5, default value =  0.75)
        power=area*eta*GHI*PR_PV; #(shadings not included)*
        return power #W
    
    #######################
    #Calculation
    #######################
    installed_wind=(5-n_PV)/eta_max_wind*rated_power_wind;
    installed_PV=n_PV/(eta_pv*PR_PV)*eta_pv*PR_PV*area_pv*1000;
    dff['Power_WT'] = (5-n_PV)/eta_max_wind*dff['Speed_hubheight'].apply(power_wind,rated_power=rated_power_wind,eta_max=eta_max_wind,rated_windspeed=rated_speed_wind,cutin_speed=cutin_speed_wind,cutout_speed=cutout_speed_wind,rho=rho_air,A=area_swept)
    dff['Power_PV'] = n_PV/(eta_pv*PR_PV)*dff['GHI'].apply(power_pv,eta=eta_pv, area=area_pv, PR=PR_PV)
    dff['Power'] = dff['Power_PV']+dff['Power_WT']
    
    dff.to_csv('RESData_option-'+str(n_PV)+'.csv', index=True)
    
    return installed_wind, installed_PV, Load

dfff = pd.DataFrame({"Option": [0, 1, 2, 3,4,5]})
dfff['installed wind'], dfff['installed PV'], dfff['Load'] = zip(*dfff['Option'].apply(RESCalc))


In [11]:
dfff

,Option,installed wind,installed PV,Load
0,0,1.163158e+07,0.0,20250000
1,1,9.305264e+06,7200000.0,20250000
2,2,6.978948e+06,14400000.0,20250000
3,3,4.652632e+06,21600000.0,20250000
4,4,2.326316e+06,28800000.0,20250000
5,5,0.000000e+00,36000000.0,20250000
